In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/")
import latexplot
latexplot.set_cmap() 
from spec_rename import SpecRename

from simsopt import mhd
from simsopt import geo
from simsopt import objectives
import matplotlib.pyplot as plt


import glob

directories = {"QI_failed":"../../freeb_12-23-01-03-52"}

In [2]:
def plot_booz_nofig(vmec, **kwargs):
    import booz_xform
    from matplotlib import cm
    vmec.run()
    sarr = [0.95]
    print("aspect", vmec.boundary.aspect_ratio())
    mpol = kwargs.pop("mpol", 32)
    ntor = kwargs.pop("ntor", 32)
    boozer = mhd.Boozer(vmec, mpol, ntor)
    boozer.register(sarr)
    boozer.run()

    if "cmap" not in kwargs:
        kwargs["cmap"] = cm.plasma
    if "fill" not in kwargs:
        kwargs["fill"] = False

    booz_xform.surfplot(boozer.bx, 0, **kwargs)


def plot_booz_spec(filename, label=None):
    if filename.startswith("input."): 
        vmec = mhd.Vmec(filename, verbose=False)
        specf = filename.replace("input.", "")
    else:
        vmec = mhd.Vmec(verbose=False)
        with SpecRename(filename) as specf:
            print(f"renamed {filename} to {specf}")
            spec = mhd.Spec(specf, tolerance=1e-10)
            vmec.boundary = spec.boundary.copy()
            # vmec.boundary = geo.SurfaceRZFourier.from_vmec_input(specf)
    plot_booz_nofig(vmec, ncontours=16)
    label = label or filename
    plt.title(f"Boozer plot for {label}")

In [ ]:

for key, directory in directories.items():
    subdirs = glob.glob(f"{directory}/mpol*")
    subdirs = sorted(subdirs)

    cols = 2
    nrows = int(np.ceil((len(subdirs)+1)/cols))
    fig, axs = plt.subplots(nrows, cols, figsize=latexplot.get_size(1, (nrows, cols)), sharex=True, sharey=True)    
    axs = axs.flatten()
    subplot = 0
    plt.sca(axs[subplot])

    if "freeb" in directory:
        # Freeboundary
        first_file = glob.glob(f"{subdirs[0]}/*_000_000000.sp.h5")[0]
        plot_booz_spec(first_file, f"mpol=0")
    else:
        first_file = glob.glob(f"{subdirs[0]}/input*_000_000000")[0]
        plot_booz_spec(first_file, f"mpol=0")
    print(first_file)
    subplot += 1
    for i, subdir in enumerate(subdirs):
        plt.sca(axs[subplot])
        res = int(subdir.split("mpol")[-1])
        if "freeb" in directory:
            latest_file = max(glob.glob(f"{subdir}/*_000_*.h5"), key=lambda x: int(x.split("_")[-1].split(".")[0]))
            plot_booz_spec(latest_file, f"mpol={res}")
        else:
            # Fixboundary
            latest_file = max(glob.glob(f"{subdir}/input*_000_*"), key=lambda x: int(x.split("_")[-1]))
            plot_booz_spec(latest_file, f"mpol={res}")

        if i < nrows*(cols-1):
            plt.xlabel("")
        if i % nrows > 0:
            plt.ylabel("")
        plt.gca().label_outer()
        subplot += 1
    latexplot.savenshow(directory+"/booz_forms")